Run an OSS-DBS input file
that fails with a geometry
to debug.


In [ ]:
import json
import logging

import netgen.occ
import numpy as np
from netgen.webgui import Draw

import ossdbs

In [ ]:
try:
    import webgui_jupyter_widgets
    from packaging.version import parse

    assert parse(webgui_jupyter_widgets.__version__) >= parse("0.2.18")
    print("Everything good!")
except AssertionError:
    print(
        "\x1b[31mYou need to update webgui_jupyter_widgets by running:"
        " \x1b[0m\npython3 -m pip install --upgrade webgui_jupyter_widgets"
    )

In [ ]:
input_file = "oss-dbs_parameters_bee.json"

In [ ]:
ossdbs.set_logger(logging.DEBUG)
with open(input_file) as fp:
    settings = json.load(fp)

In [ ]:
brain = ossdbs.generate_brain_model(settings)
Draw(brain.geometry)

In [ ]:
electrodes = ossdbs.generate_electrodes(settings)
for electrode in electrodes:
    Draw(electrode.geometry)
    if not np.isclose(electrode.encapsulation_thickness, 0):
        encapsulation = electrode.encapsulation_geometry(
            electrode.encapsulation_thickness
        )
        Draw(encapsulation)
        encapsulation = encapsulation * brain.geometry
        Draw(encapsulation)
        brain_geo = netgen.occ.Glue([brain.geometry, encapsulation])
    else:
        brain_geo = brain.geometry
    Draw(brain_geo)
    brain_f = brain_geo - electrode.geometry
    Draw(brain_f)

In [ ]:
brain_rotated = ossdbs.generate_brain_model(settings, rotate_initial_geo=True)
Draw(brain_rotated.geometry)

In [ ]:
print(brain.geometry.center)
print(brain_rotated.geometry.center)
print(brain.geometry.bounding_box)
print(brain_rotated.geometry.bounding_box)
print(brain.geometry.mass)
print(brain_rotated.geometry.mass)

In [ ]:
model_geometry = ossdbs.ModelGeometry(brain, electrodes)

In [ ]:
Draw(netgen.occ.Glue([brain.geometry, electrode.geometry]))

In [ ]:
Draw(model_geometry.geometry.shape)